# DNA Sequences Classification

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
seed = 1

## DNA Sequencing

DNA sequencing is the process of determining the nucleic acid sequence, i.e. the order of nucleotides in DNA. It includes any method or technology that is used to determine the order of the four bases: adenine, guanine, cytosine, and thymine.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTLir-DRE3gtzuuom9f9w4snEI5mEGZ6zXq8Q&usqp=CAU" style="width:40%;"/>

Nucleic acids consist of a chain of linked units called **nucleotides**. Each nucleotide consists of three subunits: a phosphate group and a sugar (deoxyribose in DNA) make up the backbone of the nucleic acid strand, and attached to the sugar is one of a set of nucleobases. The possible letters are A, C, G, and T, representing the four nucleotide bases of a DNA strand, adenine, cytosine, guanine, thymine, covalently linked to a phosphodiester backbone. In the typical case, the sequences are printed abutting one another without gaps, as in the sequence AAAGTCTGAC, read left to right in the 5' to 3' direction.

<img src="https://sciencenotes.org/wp-content/uploads/2020/11/Parts-of-a-Nucleotide.png" style="width:30%;"/>

A **base pair** is a fundamental unit of double-stranded nucleic acids consisting of two nucleobases bound to each other by hydrogen bonds. They form the building blocks of the DNA double helix and contribute to the folded structure of both DNA and RNA.

A **promoter** is a sequence of DNA needed to turn a gene on or off. The process of transcription is initiated at the promoter. Usually found near the beginning of a gene, the promoter has a binding site for the enzyme used to make a messenger RNA (mRNA) molecule.

<img src="https://www.genome.gov/sites/default/files/tg/en/illustration/promoter.jpg" style="width:40%;"/>

# Data Set

The [Molecular Biology (Promoter Gene Sequences) Data Set](https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/promoter-gene-sequences/) consists of 106 Escherichia Coli DNA sequences, with 57 sequential nucleotides (base-pairs) each. The attributes are the following:

* Class: + = promoter, - = non-promoter
* The instance name (non-promoters named by position in the 1500-long nucleotide sequence provided by Prof. Tom Record).
* The DNA sequence, filled by one of {a, g, t, c}.

In [3]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/promoter-gene-sequences/promoters.data'
data = pd.read_csv(url, names=['Class', 'id', 'Sequence'])

In [4]:
print(f'Shape: {data.shape}')
data.head()

Shape: (106, 3)


,Class,id,Sequence
0,+,S10,\t\ttactagcaatacgcttgcgttcggtggttaagtatgtataat...
1,+,AMPC,\t\ttgctatcctgacagttgtcacgctgattggtgtcgttacaat...
2,+,AROH,\t\tgtactagagaactagtgcattagcttatttttttgttatcat...
3,+,DEOP2,\taattgtgatgtgtatcgaagtgtgttgcggagtagatgttagaa...
4,+,LEU1_TRNA,\ttcgataattaactattgacgaaaagctgaaaaccactagaatgc...


## Data Preprocessing

In [5]:
data['Sequence'] = data['Sequence'].str.replace(r'\t', '')

<ipython-input-5-116d3fecf603>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Sequence'] = data['Sequence'].str.replace(r'\t', '')


In [6]:
dna_sequences_df = data['Sequence'].apply(lambda x: pd.Series(list(x)))
dna_sequences_df['Class'] = data['Class']
dna_sequences_df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,Class
0,t,a,c,t,a,g,c,a,a,t,...,g,c,t,t,g,t,c,g,t,+
1,t,g,c,t,a,t,c,c,t,g,...,c,a,t,c,g,c,c,a,a,+
2,g,t,a,c,t,a,g,a,g,a,...,c,a,c,c,c,g,g,c,g,+
3,a,a,t,t,g,t,g,a,t,g,...,a,a,c,a,a,a,c,t,c,+
4,t,c,g,a,t,a,a,t,t,a,...,c,c,g,t,g,g,t,a,g,+


In [10]:
dna_sequences_hot_encoded_df = pd.get_dummies(dna_sequences_df)
dna_sequences_hot_encoded_df.drop(columns=['Class_-'], inplace=True)
dna_sequences_hot_encoded_df.rename(columns = {'Class_+': 'Class'}, inplace = True)

In [11]:
X = np.array(dna_sequences_hot_encoded_df.drop(['Class'], axis=1))
y = np.array(dna_sequences_hot_encoded_df['Class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

## Train Models with Cross-Validation

In [16]:
names = ['Nearest Neighbors', 'Gaussian Process', 'Decision Tree', 'Random Forest', 'Neural Net', 'AdaBoost', 'Naive Bayes', 'SVM Linear', 'SVM RBF', 'SVM Sigmoid']

models = [
    KNeighborsClassifier(n_neighbors = 3),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=.5, max_iter=500),
    AdaBoostClassifier(),
    GaussianNB(),
    SVC(kernel='linear'), 
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid')
]

results = []

for model in models:
    kfold = KFold(n_splits=10)
    scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(scores)
    
results = np.array(results)

In [30]:
print('Cross Validation Mean Accuracy\n')
for name, mean in zip(names, results.mean(axis=1)):
    print(f'{name}: {mean:.2f}')

Cross Validation Mean Accuracy

Nearest Neighbors: 0.84
Gaussian Process: 0.87
Decision Tree: 0.77
Random Forest: 0.69
Neural Net: 0.88
AdaBoost: 0.93
Naive Bayes: 0.84
SVM Linear: 0.85
SVM RBF: 0.89
SVM Sigmoid: 0.90


## Evaluation

In [33]:
for name, model in zip(names, models):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'{name}: {accuracy_score(y_test, y_pred):.2f}')

Nearest Neighbors: 0.78
Gaussian Process: 0.89
Decision Tree: 0.74
Random Forest: 0.63
Neural Net: 0.93
AdaBoost: 0.85
Naive Bayes: 0.93
SVM Linear: 0.96
SVM RBF: 0.93
SVM Sigmoid: 0.93
